# Kurvanpassning av brusig data

Målet är att se hur en minsta-kvadrat-anpassning av Python går till och hur den kan återskapa den sanna kurvan bakom våra mätpunkter. Vi skapar först en sann kurva av vår modellfunktion, sedan räknar vi ut "mätpunkter" som vi förse med slumpmässiga avvikelser för att sedan försöka återskapa originalfunktionen.

Först igen våra importerade bibliotek.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 9]
import scipy.optimize as opt
import numpy as np
from IPython.display import display, Math, Markdown

Funktionen som vi utgår ifrån är en tredjegrads polynom<br/>
$f(x)=A x^3+B x^2 + C x + D$<br/>
Parametrarna $A$, $B$, $C$ och $D$ måste stå med i funktionsdeklarationen.

In [ ]:
def funktion(x, A, B, C, D):
    return A*x**3 + B*x**2 + C*x + D

Med hjälp av `np.arange()` skapar vi en lista med x-koordinater från -10 till 10 i steg på 0.01

In [ ]:
x = np.arange(-10, 10, 0.01)
print(x)

Nu definieras de sanna funktionsvärderna med funktionen
$f(x)=x^3 + 20 x^2 + 30 x + 40$

In [ ]:
origParams = [1, 20, 30, 40]

y_sann = funktion(x, *origParams)

Hur ser den sanna kurvan ut?

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y_sann, "k-", label = "original", linewidth=2)

Nu skapar vi _brus_ med hjälp av normalfördelade slumptal med<br/>
$\overline{y}=0.0$ och $\sigma=200$

In [ ]:
brus = np.random.normal(loc=0, scale=200, size=x.size)

fig, ax = plt.subplots()
ax.plot(x, brus, "b.", label = "brus", markersize = 1)
ax.axhline(y=np.mean(brus), color="red")
ax.axhspan(ymin=-1*np.std(brus), ymax=+1*np.std(brus), color="yellow", alpha=0.7)
ax.axhspan(ymin=-2*np.std(brus), ymax=+2*np.std(brus), color="yellow", alpha=0.4)
ax.axhspan(ymin=-3*np.std(brus), ymax=+3*np.std(brus), color="yellow", alpha=0.2)

Nu adderar vi brus-vektorn till vektorn med våra mätpunkter

In [ ]:
y_brus = y_sann + brus

Och sedan kollar vi hur mätpunkterna med brus ser ut:

In [ ]:
fig, ax = plt.subplots()
ax.plot(x, y_brus, "b.", label = "med brus", markersize = 3)
ax.plot(x, y_sann, "k-", label = "original", linewidth=1)

Till kurvanpassningen måste vi börja med uppskattade/gissade startvärden fär våra parametrar. Python letar sedan efter den bästa kurvanpassningen med dessa startvärden som utgångspunkt.<br/>
Låt oss även kolla hur kurvan med våra stratvärden som parameter ser ut:

In [ ]:
guessParams = [0, 0, 0, 0]

fig, ax = plt.subplots()
ax.plot(x, y_brus, "b.", label = "med brus", markersize = 3)
ax.plot(x, y_sann, "k-", label = "original", linewidth=1)
ax.plot(x, funktion(x, *guessParams), "m-", label = "gissning", linewidth=2)
ax.legend()

Och nu startar vi kurvanpassningen med `opt.curve_fit()`.<br/>
Genom att välja `verbose=2` får vi uppdateringar under anpassningsprocessen.

In [ ]:
optimParams, pcov = opt.curve_fit(f=funktion, xdata=x, ydata=y_brus, p0=guessParams, method="trf", verbose=2)

Sista raden av meddelanden från `opt.curve_fit()` ger oss lite information om Python lyckades eller misslyckades. Om den lyckades så kan vi titta på resultaten och jämföra med vår sanna funktion:

In [ ]:
print("anpassad: A={:8g}, B={:8g}, C={:8g} D={:8g}".format(*optimParams))
print("sannt:    A={:8g}, B={:8g}, C={:8g} D={:8g}".format(*origParams))

display(Markdown(r"anpassad:      $f(x) = {:8g} x^3 {:+8g} x^2 {:+8g} x {:+8g}$".format(*optimParams)))
display(Markdown(r"sann funktion: $f(x) = {:8g} x^3 {:+8g} x^2 {:+8g} x {:+8g}$".format(*origParams)))

Och hur ser det ut som graf?

dessa kommandon använde vi ovan:<br/>
`fig, ax = plt.subplots()`<br/>
`ax.plot(x, y_brus, "b.", label = "med brus", markersize = 3)`<br/>
`ax.plot(x, y_sann, "k-", label = "original", linewidth = 1)`<br/>
`ax.plot(x, funktion(x, *guessParams), "m-", label = "gissning", linewidth = 2)`<br/>
`ax.plot(x, funktion(x, *optimParams), "r-", label = "anpassad", linewidth = 5)`<br/>
`ax.legend()`<br/>


In [ ]:
# gör din egen plot här